# Python Advanced

## Generator

In [10]:
def pow2():
    n = 2
    while n < 1000:
        yield n
        n *= 2

print([i for i in pow2()])

a = pow2()

print(next(a))
print(next(a))
print(next(a))

[2, 4, 8, 16, 32, 64, 128, 256, 512]
2
4
8


## [Coroutine](https://www.youtube.com/watch?v=7AoANOGIDuM)

In [6]:
# 呼叫 next() 時會跑到 coro 裡的下一個 yield
# 然後可以用 send 把值傳進正在跑的函數裡，同時 send 也會 return yield 的結果

def coro():
    step = 0
    while True:
        received = yield step
        step += 1
        print(f'Received: {received}')

c = coro()
next(c)               # important! get to the first yield
print(c.send(100))

Received: 100
1


## Decorator 

* 寫的很好的 [RealPython tutorial](https://realpython.com/primer-on-python-decorators/#stateful-decorators)，整篇看完了但沒時間作筆記
* ```@debug``` 印下函數的 input/output，可以用寫 recursive 的時候 debug
* [classes as decorators](https://realpython.com/primer-on-python-decorators/#classes-as-decorators)，implement ```__init__``` 和 ```__call__```，可以存狀態，例如 lru_cache

### General Pattern (No Argument)

In [7]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

### Decorator fix_seed

In [367]:
# fix_seed：固定 seed = 0 版本。離開函數 seed 會還原成 None

import numpy as np
import functools

def fix_seed(fnc):
    @functools.wraps(fnc)
    def wrapper_fix_seed(*args, **kargs):
        np.random.seed(0)
        res = fnc(*args, **kargs)
        np.random.seed()
        return res
    return wrapper_fix_seed

@fix_seed
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())

0.5488135039273248
0.6161167995056092


In [377]:
# 接受 argument 版本，但變成一定要指定 seed

import numpy as np
import functools

def fix_seed(seed=0):
    def decorator_fix_seed(fnc):
        @functools.wraps(fnc)
        def wrapper_fix_seed(*args, **kargs):
            np.random.seed(seed)
            res = fnc(*args, **kargs)
            np.random.seed()
            return res
        return wrapper_fix_seed
    return decorator_fix_seed

@fix_seed(100)
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())

0.5434049417909654
0.3289099673526439


In [6]:
# 可以指定也可以不指定。若不指定 seed 預設為 0。若要指定一定要寫 seed=

# 有指定 seed 的時候相當於 printRand = fix_seed(seed=0)(printRand)，所以 _func 是 None
# 不指定 seed 的時候則變成 printRand = fix_seed(printRand)          把 function 傳進去

import numpy as np
import functools

def fix_seed(_func=None, *, seed=0):
    def decorator_fix_seed(func):
        @functools.wraps(func)
        def wrapper_fix_seed(*args, **kwargs):
            np.random.seed(seed)
            res = func(*args, **kwargs)
            np.random.seed()
            return res
        return wrapper_fix_seed

    if _func:
        return decorator_fix_seed(_func)
    else:
        return decorator_fix_seed

    
# @fix_seed(0)   # TypeError: 'int' object is not callable
# @fix_seed(seed=0)
@fix_seed
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())    

0.5488135039273248
0.13056825103667768
